In [17]:
using Pkg
Pkg.activate(@__DIR__)
using SummationByParts, LinearAlgebra
using SummationByParts.Cubature, SummationByParts.SymCubatures
using Latexify

  Activating project at `~/Documents/codes/SummationByParts.jl/examples`


# Construct SBP Operators

Okay, we have derived some existing or new quadrature rules. How do we proceed to construct SBP operators?

## SBP Operators on Triangles (with LG or LGL facet nodes)

Consider the quadrature rule we derived for SBP diagonal-E operators.

In [18]:
qf = 11; # the facet quadrature must be qf=q+mod(q,2)+1
cub,_ = SummationByParts.Cubature.quadrature(qf, internal=false); # get LGL nodes for the facet nodes
xedge = cub.params;

In [19]:
cub_tri, vtx_tri = SummationByParts.deriveTriCubatureDiagE(q=9,
                                                    vertices=true, 
                                                    midedges=true, 
                                                    numS21=3, 
                                                    numedge=2, 
                                                    numS111=1, 
                                                    centroid=false,
                                                    delta1=1e-2, delta2=1e-2, xinit = [], xedge = xedge, verbose=false)

No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
----------------------------------------------------------------------------------------------
iter_pso = 8800:  iter_lma = 1043:  nperturb_pso = 36:  res norm = 3.7868127614616725e-15
----------------------------------------------------------------------------------------------


(TriSymCub{Float64}(7, 8, 33, true, true, false, 2, 3, 1, [0, 5, 3], [0.19981462157882743, 0.5111735695695017, 0.9090035832488029, 0.9151119481392835, 0.7344243967353571, 1.281990991479007, 0.1467235719409893], [0.001928119141153443, 0.02299604741162223, 0.09839509446752868, 0.18658872403391172, 0.09275708200964641, 0.016522971823876034, 0.017823898017816347, 0.09765392995970967]), [-1.0 -1.0; 1.0 -1.0; -1.0 1.0])

To build SBP diagonal-E opertor using the above quadrature, we proceed as follows:

In [20]:
w, Q, E = SummationByParts.buildoperators(cub_tri, vtx_tri, 5, vertices=true)

([0.001928119141153443, 0.001928119141153443, 0.001928119141153443, 0.02299604741162223, 0.02299604741162223, 0.02299604741162223, 0.09839509446752868, 0.09839509446752868, 0.09839509446752868, 0.18658872403391172  …  0.017823898017816347, 0.017823898017816347, 0.017823898017816347, 0.017823898017816347, 0.09765392995970967, 0.09765392995970967, 0.09765392995970967, 0.09765392995970967, 0.09765392995970967, 0.09765392995970967], [-0.023809523809523774 -0.00207602253104634 … 0.01428398209884167 -0.022385315634735274; 0.00207602253104634 0.023809523809523774 … 0.00023869670330554997 -0.0003172963902544196; … ; -0.01428398209884167 -0.00023869670330554997 … 0.0 -0.03193735590894055; 0.022385315634735274 0.0003172963902544196 … 0.03193735590894055 0.0;;; -0.023809523809523774 -0.0010380112655232017 … 0.01429386956392125 -0.025517208150105865; 0.0010380112655231917 0.0 … 0.00055599309355983 -0.0005559930935596127; … ; -0.01429386956392125 -0.00055599309355983 … 0.0 -0.06387471181788076; 0.0

Now we can access all operators as follows:

In [21]:
Qx = Q[:,:,1];
Qy = Q[:,:,2];
H = diagm(w);
Ex = E[:,:,1];
Ey = E[:,:,2];
Dx = inv(H)*Qx;
Dy = inv(H)*Qy;
Sx = Qx - 0.5.*Ex;
Sy = Qy - 0.5.*Ey;

We can check if the SBP property is satisfied.

In [22]:
norm(Qx+Qx' - Ex)

1.1294959466037524e-17

In [23]:
norm(Qy+Qy' - Ey)

1.1294959466037524e-17

## SBP Operators on Tetrahedra

Consider the quadrature rule of degree 6 that we have derived previously to construct a degree $p=3$ SBP diagonal-E operator. 

In [24]:
q=6;
p=3;
qf=2*p;

In [25]:
cub_facet, vtx_facet = SummationByParts.deriveTriCubatureGamma(q=qf,
                                                       vertices=true,
                                                       midedges=false,
                                                       numS21=2,
                                                       numedge=1,
                                                       numS111=0,
                                                       centroid=false,
                                                       xinit=[],
                                                       delta1=1e-3,delta2=1e-1,verbose=false)

No solution found after a 200 LMA iterations.
----------------------------------------------------------------------------------------------
iter_pso = 3200:  iter_lma = 212:  nperturb_pso = 46:  res norm = 1.7233264267057687e-15
----------------------------------------------------------------------------------------------


(TriSymCub{Float64}(3, 4, 15, true, false, false, 1, 2, 0, [0, 3, 1], [0.8506802519794944, 0.23722737279318576, 0.6922540583740083], [0.014260718614408951, 0.3303589772911329, 0.20376930605390367, 0.05913883235361056]), [-1.0 -1.0; 1.0 -1.0; -1.0 1.0])

In [26]:
xedge = cub_facet.params
cub_tet, vtx_tet = SummationByParts.deriveTetCubatureDiagE(q=q,
                                                   vertices=true,
                                                   numS31=1,
                                                   midedges=false, 
                                                   numS22=1,
                                                   numfaceS21=2, 
                                                   numedge=1, 
                                                   numS211=0,
                                                   numfaceS111=0, 
                                                   facecentroid=false,
                                                   numS1111=0,
                                                   centroid=true,
                                                   xinit=[],
                                                   xedge=xedge,
                                                   delta1=1e-3,
                                                   delta2=1e-1,
                                                   verbose=false)

----------------------------------------------------------------------------------------------
iter_pso = 800:  iter_lma = 12:  nperturb_pso = 2:  res norm = 4.0790928673662795e-14
----------------------------------------------------------------------------------------------


(TetSymCub{Float64}(5, 7, 51, true, false, true, false, 1, 2, 0, 1, 1, 0, 0, [1, 2, 1, 3, 0], [0.39314846687716914, 0.2578639185632089, 0.8506802519794944, 0.23722737279318576, 0.6922540583740083], [0.0008311992138366838, 0.06707451628396319, 0.08700624630787598, 0.02528103240642225, 0.014487003956738086, 0.004650827385021294, 0.006646628516701348]), [-1.0 -1.0 -1.0; 1.0 -1.0 -1.0; -1.0 1.0 -1.0; -1.0 -1.0 1.0])

Now, we can simply construct the an SBP operator of degree $p=3$ as follows:

In [27]:
# We must provide the facet operator type, currently the availabe types are, :Omega and :DiagE
# To enforce using the facet quadrature we derived, we must also provide facetcub and facetvtx; 
# otherwise, available default facet operators saved in the code will be used
w, Q, E = SummationByParts.buildoperators(cub_tet, vtx_tet, p, faceopertype=:DiagE, facecub=cub_facet, facevtx=vtx_facet)

([0.0008311992138366838, 0.0008311992138366838, 0.0008311992138366838, 0.0008311992138366838, 0.06707451628396319, 0.06707451628396319, 0.06707451628396319, 0.06707451628396319, 0.08700624630787598, 0.08700624630787598  …  0.004650827385021294, 0.004650827385021294, 0.004650827385021294, 0.004650827385021294, 0.004650827385021294, 0.004650827385021294, 0.004650827385021294, 0.004650827385021294, 0.004650827385021294, 0.006646628516701348], [-0.007130359307204476 -0.0011963999592267448 … -5.298891963682456e-5 0.0032641516168219666; 0.0011963999592267448 0.007130359307204476 … 5.298891963686027e-5 -0.0032641516168218716; … ; 5.298891963682456e-5 -5.298891963686027e-5 … 0.0 -1.2464381309970012e-17; -0.0032641516168219666 0.0032641516168218716 … 1.2464381309970012e-17 0.0;;; -0.007130359307204476 -0.0005981999796133577 … -0.002704982226496509 0.003264151616821969; 0.0005981999796133577 0.0 … -0.0026519933068596922 6.27490369179953e-17; … ; 0.002704982226496509 0.0026519933068596922 … 0.029

Then, we access all the remaining operators as:

In [28]:
Qx = Q[:,:,1];
Qy = Q[:,:,2];
Qz = Q[:,:,3];
H = diagm(w);
Ex = E[:,:,1];
Ey = E[:,:,2];
Ez = E[:,:,3];
Dx = inv(H)*Qx;
Dy = inv(H)*Qy;
Dz = inv(H)*Qz;
Sx = Qx - 0.5.*Ex;
Sy = Qy - 0.5.*Ey;
Sz = Qz - 0.5.*Ez;

Check if the SBP property is satisfied:

In [29]:
norm(Qx+Qx' - Ex)

0.0

In [30]:
norm(Qy+Qy' - Ey)

0.0

In [31]:
norm(Qz+Qz' - Ez)

0.0